In [1]:
import azureml
from IPython.display import display, Markdown
from azureml.core import  Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration


from model_drift import settings, helpers

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

Azure ML SDK Version:  1.38.0


In [2]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)

In [3]:
env_name = "monitoring"

# Name experiement
experiment_name = 'train-model'
compute_target = "nc24-uswest2"

pretrained =  "pretrained-chexpert-epoch-4-step-29844.ckpt"

# Input Dataset
dataset = Dataset.get_by_name(ws, name='padchest')

#Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
pytorch_env.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# Base Params
args = {
 'data_folder': dataset.as_named_input('dataset').as_mount(),
 'run_azure': 1,
 'batch_size': 6,
 'output_dir': './outputs',
 'pretrained': pretrained,
 'num_workers': -1,
 'max_epochs':  24,
 'progress_bar_refresh_rate': 25,
 'log_every_n_steps':  25 ,
 'flush_logs_every_n_steps':  25,
 'accelerator': 'ddp',
 'freeze_backbone': 1,
 'frontal_only': 1,
 'num_sanity_val_steps': 0
 }


hyperparameters = {
    "accumulate_grad_batches": 1,
    "batch_size": 48/4, # batch repeated across GPUs, if you are using a different # of GPUs please upcate
    "freeze_backbone": 1,
    "frontal_only": 1,
    "learning_rate": 0.001,
    "max_epochs": 48,
    "step_size": 12
}

args.update(hyperparameters)

config = ScriptRunConfig(
    source_directory = str(project_dir), 
    script = "scripts/finetune/train.py",
    arguments=helpers.argsdict2list(args),
)

config.run_config = run_config

for param, value in args.items():
    print(f"{param}: {value}")
    
config.run_config.target = compute_target
run = exp.submit(config)
display(Markdown(f"""
- Environment: {pytorch_env.name}
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))

data_folder: <azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x000002DE6EAAE808>
run_azure: 1
batch_size: 12.0
output_dir: ./outputs
pretrained: pretrained-chexpert-epoch-4-step-29844.ckpt
num_workers: -1
max_epochs: 48
progress_bar_refresh_rate: 25
log_every_n_steps: 25
flush_logs_every_n_steps: 25
accelerator: ddp
freeze_backbone: 1
frontal_only: 1
num_sanity_val_steps: 0
accumulate_grad_batches: 1
learning_rate: 0.001
step_size: 12



- Environment: monitoring
- Experiment: [train-model-padchest](https://ml.azure.com/experiments/train-model-padchest?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [sweet_jelly_3x6wmp5j](https://ml.azure.com/runs/train-model-padchest_1644446815_7e2d14fd?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: nc24-uswest2
